# Probing part 1

Vantage points will probe either the targets themselves (step 2) or other addresses in the same /24 prefix (step 2).

Vantage points are only the anchors.  
As always, targets are the anchors.  

This notebook is an implementation of the million scale method. Check the paper for more information.  
To do after create_datasets.ipynb

In [1]:
import uuid

from logger import logger
from scripts.utils.file_utils import load_json
from scripts.utils.measurement_utils import (
    load_targets,
    load_vps,
    get_measurement_config,
    save_measurement_config,
    get_target_prefixes,
    ping_prefixes,
    ping_targets,
)
from default import (
    USER_ANCHORS_FILE,
    USER_HITLIST_FILE,
    MEASUREMENT_CONFIG_PATH,
)

# will define the number of vps and targets to use
NB_TARGETS = 2
NB_VPS = 4

## Load targets and vps dataset

In [2]:
targets = load_targets(USER_ANCHORS_FILE, nb_target=NB_TARGETS)
vps = load_vps(USER_ANCHORS_FILE, nb_vps=NB_VPS)

# every anchors /24 subnet
target_addrs = [t["address_v4"] for t in targets]
target_prefixes = get_target_prefixes(target_addrs)

# responsive IP addresses in all /24 prefixes
targets_per_prefix = load_json(USER_HITLIST_FILE)

logger.info(f"nb targets: {len(targets)}")
logger.info(f"nb_vps : {len(vps)}")

## Generate measurement config

This configuration is used to retrieve all measurements results from RIPE Atlas using their API.

In [3]:
# measurement configuration for retrieval
experiment_uuid = str(uuid.uuid4())
target_measurement_uuid = str(uuid.uuid4())
prefix_measurement_uuid = str(uuid.uuid4())

config_file_path = MEASUREMENT_CONFIG_PATH / f"{experiment_uuid}.json"

logger.info(f"Starting experiment with uuid : {experiment_uuid}")

measurement_config = get_measurement_config(
    experiment_uuid=experiment_uuid,
    target_measurement_uuid=target_measurement_uuid,
    prefix_measurement_uuid=prefix_measurement_uuid,
    targets=targets,
    target_prefixes=target_prefixes,
    vps=vps,
)

save_measurement_config(measurement_config, config_file_path)

2024-10-04 13:13:06::INFO:root:3171606573:: Starting experiment with uuid : c78efe35-8089-41a9-9206-ac7bac4a8a68


# Step 1: probing each target prefixes

## Probe target prefixes
WARNING : Time consumming section

In [4]:
ping_prefixes(
    measurement_uuid=prefix_measurement_uuid,
    measurement_config=measurement_config,
    target_prefixes=target_prefixes,
    targets_per_prefix=targets_per_prefix,
    vps=vps,
)

save_measurement_config(measurement_config, config_file_path)

2024-10-04 13:13:06::INFO:root:measurement_utils:: No cached results available
2024-10-04 13:13:06::INFO:root:measurement_utils:: Starting measurements dd2e9428-762d-4353-99ca-613057d430a3 with parameters: dry_run=False; nb_targets=2; nb_vps=4.
2024-10-04 13:13:06::INFO:root:ping_and_traceroute_classes:: measurement tag: dd2e9428-762d-4353-99ca-613057d430a3 : started measurement id : 79942232
2024-10-04 13:13:08::INFO:root:ping_and_traceroute_classes:: measurement tag: dd2e9428-762d-4353-99ca-613057d430a3 : started measurement id : 79942233
2024-10-04 13:13:08::INFO:root:ping_and_traceroute_classes:: measurement tag: dd2e9428-762d-4353-99ca-613057d430a3 : started measurement id : 79942234
2024-10-04 13:13:08::INFO:root:ping_and_traceroute_classes:: measurement tag: dd2e9428-762d-4353-99ca-613057d430a3 : started measurement id : 79942235
2024-10-04 13:13:09::INFO:root:ping_and_traceroute_classes:: measurement tag: dd2e9428-762d-4353-99ca-613057d430a3 : started measurement id : 79942236


# Step 2: probing each target

## Probe targets
WARNING : Time consumming section

In [6]:
# measurement configuration for retrieval
ping_targets(
    measurement_uuid=target_measurement_uuid,
    measurement_config=measurement_config,
    targets=targets,
    vps=vps,
    use_cache=False,
)
save_measurement_config(measurement_config, config_file_path)

2024-10-04 13:13:11::INFO:root:measurement_utils:: Starting measurements 6796bfe3-7137-43f1-9f9f-71e0a141157d with parameters: dry_run=False; nb_targets=6; nb_vps=4.
2024-10-04 13:13:12::INFO:root:ping_and_traceroute_classes:: measurement tag: 6796bfe3-7137-43f1-9f9f-71e0a141157d : started measurement id : 79942244
2024-10-04 13:13:12::INFO:root:ping_and_traceroute_classes:: measurement tag: 6796bfe3-7137-43f1-9f9f-71e0a141157d : started measurement id : 79942245
2024-10-04 13:13:12::INFO:root:ping_and_traceroute_classes:: measurement tag: 6796bfe3-7137-43f1-9f9f-71e0a141157d : started measurement id : 79942246
2024-10-04 13:13:13::INFO:root:ping_and_traceroute_classes:: measurement tag: 6796bfe3-7137-43f1-9f9f-71e0a141157d : started measurement id : 79942247
2024-10-04 13:13:13::INFO:root:ping_and_traceroute_classes:: measurement tag: 6796bfe3-7137-43f1-9f9f-71e0a141157d : started measurement id : 79942248
2024-10-04 13:13:14::INFO:root:ping_and_traceroute_classes:: measurement tag: 6

## Retrieve prefix results
WARNING : Time consuming section

Note: it might take some time before measurement results are available through RIPE API. If no results are available, retry after a few minutes (or hours, it might really depends on the probe itself).

In [7]:
from logger import logger
from scripts.utils.file_utils import load_json
from scripts.utils.measurement_utils import (
    retrieve_results,
    insert_prefix_results,
    insert_target_results,
)
from default import (
    PREFIX_MEASUREMENT_RESULTS,
    TARGET_MEASUREMENT_RESULTS,
)

# will define the number of vps and targets to use
NB_TARGETS = 2
NB_VPS = 4

In [8]:
measurement_config = load_json(config_file_path)
logger.info(measurement_config)

2024-10-04 13:13:14::INFO:root:3539837011:: {'experiment_uuid': 'c78efe35-8089-41a9-9206-ac7bac4a8a68', 'status': 'ongoing', 'start_time': '2024-10-04 13:13:06.112516', 'end_time': None, 'is_dry_run': False, 'nb_targets': 2, 'nb_vps': 4, 'description': 'measurements from a set of vps towards all targets/target prefixes', 'af': 4, 'target_measurements': {'measurement_uuid': '6796bfe3-7137-43f1-9f9f-71e0a141157d', 'targets': ['103.196.37.98', '195.246.236.1', '77.220.233.1', '185.230.79.16', '185.34.2.114', '217.25.179.62'], 'vps': {'77.220.233.1': {'id': 6824, 'address_v4': '77.220.233.1', 'asn_v4': 42699, 'country_code': 'DE', 'geometry': {'type': 'Point', 'coordinates': [13.7285, 51.0395]}}, '185.230.79.16': {'id': 7122, 'address_v4': '185.230.79.16', 'asn_v4': 204515, 'country_code': 'FR', 'geometry': {'type': 'Point', 'coordinates': [2.1585, 48.7085]}}, '185.34.2.114': {'id': 6798, 'address_v4': '185.34.2.114', 'asn_v4': 36236, 'country_code': 'AE', 'geometry': {'type': 'Point', 'co

In [9]:
prefix_measurement_uuid = measurement_config["prefix_measurements"]["measurement_uuid"]

logger.info(f"retrieving results for measurement ids: {prefix_measurement_uuid}")

# sometimes, not all probes give output, reduce timeout if you do not want to wait for too long
response = retrieve_results(prefix_measurement_uuid, PREFIX_MEASUREMENT_RESULTS)

insert_prefix_results(response)

2024-10-04 13:13:14::INFO:root:1680719454:: retrieving results for measurement ids: dd2e9428-762d-4353-99ca-613057d430a3
2024-10-04 13:13:15::INFO:root:measurement_utils:: nb measurements retrieved: 0 for measurement_uuid : dd2e9428-762d-4353-99ca-613057d430a3


UnboundLocalError: local variable 'result' referenced before assignment

## Retrieve traget results
WARNING : Time consumming section

Note: it might take some time before measurement results are available through RIPE API. If no results are available, retry after a few minutes (or hours, it might really depends on the probe itself).

In [13]:
target_measurement_uuid = measurement_config["target_measurements"]["measurement_uuid"]

logger.info(f"retrieving results for measurement ids: {target_measurement_uuid}")

response = retrieve_results(target_measurement_uuid, TARGET_MEASUREMENT_RESULTS)

insert_target_results(response)

2024-10-03 18:08:53::INFO:root:3802694766:: retrieving results for measurement ids: 18020ef4-fcc5-410b-9eb1-9ab3a18dd3a3
2024-10-03 18:08:53::INFO:root:measurement_utils:: nb measurements retrieved: 20 for measurement_uuid : 18020ef4-fcc5-410b-9eb1-9ab3a18dd3a3
2024-10-03 18:08:53::INFO:root:measurement_utils:: Target measurements successfully inserted in table : user_vps_to_target
